# 尋找均線交易的最優組合。
抓出使用均線的最佳買入點、最佳賣出點組合。
此處，先固定買入點、賣出點的關係相同。讓運算時間降低。

In [1]:
import pandas, os, re, numpy
import talib

# 路徑函數
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

# pandas 所使用之欄位轉換函數
def real_profitability(vars) : 
    real_profitability = 100*(vars) - (0.1425*0.65 + 0.3) # 零股手續費。
    return real_profitability

# 回測函數
def return_test(df, indicator_1, indicator_2) : # MA_1、MA_2，都是字串。
    # 建立一個儲存買賣資訊的 list，內部每個元素都用 dict 儲存。格式 : {"買入價格" : buy_value ,"賣出價格" : sale_value}
    Order_list = []
    sale_index = -1
    for index, column_data in df.iterrows() : # 鄵，少一個存放目前是否有買入資料的 list 幹妳量，難怪會有一這麼多交易次數。
        if index > sale_index : 
            pass
        else : 
            continue
        
        # 計算買入點、售出點。
        try :  # 5_MA在前四個欄位，是遺漏值，此時無法比較大小，會報錯。 
            if column_data[indicator_1] > column_data[indicator_2] : 
                buy_value = column_data["stock_price"]
            else : 
                continue
            
            # 再次遍歷整個 data，抓出賣出點。
            for index_2, column_data_2 in df.iterrows() :
                if index_2 > index : 
                    if column_data_2[indicator_1] < column_data_2[indicator_2] : 
                        sale_value = column_data_2["stock_price"]
                        sale_index = index_2 # 避免重複交易，透過記住賣出點的 index，讓下一輪的買入點，必定要在這個賣出點後面。

                        # 儲存買入、賣出資訊。
                        Order_list.append({"buy_value" : buy_value, "sale_value" : sale_value, "profitability" : (sale_value - buy_value)/buy_value
                            , "timestamp" : column_data["timestamp"]
                            })
                        break
                    else : 
                        continue
                else : 
                    continue

        except : 
            pass

    # 買賣資訊 dataframe化，用於後續串接。
    df_Order =  pandas.DataFrame(Order_list)

    # 扣除交易手續費，以 永豐金為例，0.1425%為交易手續費。交易稅，個股為 0.3%，當充為 0.15%。
    # 創建真實獲利率函數。
    df_Order["real_profitability"] = df_Order["profitability"].map(real_profitability)
    result = {"real_profitability" : df_Order["real_profitability"].sum(), "indicator_1" : indicator_1, "indicator_2" : indicator_2}
    return result

In [ ]:
df = pandas.read_csv(os_path(["database", "APPL_stock_2023.6.12_2023.7.13.csv"]))
result_list = []
error_list = []
for i in range(2,20) :
    for i2 in range(5, 60) : 
        if i > i2 : 
            continue
        else : 
            pass
        
        df_MA = df.copy()
        df_MA[f"{i}"] = talib.SMA(df_MA.stock_price, timeperiod= i)
        df_MA[f"{i2}"] = talib.SMA(df_MA.stock_price, timeperiod= i2)
        try : 
            result = return_test(df_MA, f"{i}", f"{i2}")
            result_list.append(result)
        except : 
            error = {"indicator_1" : f"{i}", "indicator_2" : f"{i2}"}

df_result = pandas.DataFrame(result_list)
df_result[df_result["real_profitability"] == df_result["real_profitability"].max()]
# df_result.to_csv(os_path(["output", "最佳均線獲利解_買賣同.csv"]), index= False)